In [2]:
pip install keras-tuner


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
# For reading our dataset
import pandas as pd
import numpy as np

# For visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pickle

# For warnings
import warnings
warnings.filterwarnings(action='ignore')

# Importing necessary packages for building and tuning our model
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, plot_confusion_matrix, classification_report, confusion_matrix

from collections import Counter
from imblearn.under_sampling import NearMiss               # For Undersampling
from imblearn.over_sampling import RandomOverSampler       # For Oversampling
from imblearn.combine import SMOTETomek                    # For Both Undersampling & Oversampling
from sklearn.model_selection import train_test_split
#from tensorflow import keras
#from tensorflow.keras import layers
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
#from keras_tuner.tuners import RandomSearch

# For checking acuracy
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [7]:
dataML = pd.read_csv("train ML.csv")

In [8]:
dataML.head()

,Unnamed: 0,id,owner_1_score,RATE_owner_1,CAP_AMOUNT_owner_1,PERCENT_OWN_owner_1,owner_2_score,RATE_owner_2,CAP_AMOUNT_owner_2,PERCENT_OWN_owner_2,...,INPUT_VALUE_ID_FOR_avg_net_deposits,RATE_ID_FOR_avg_net_deposits,INPUT_VALUE_ID_FOR_industry_type,RATE_ID_FOR_industry_type,INPUT_VALUE_owner_4,RATE_owner_4,CAP_AMOUNT_owner_4,PERCENT_OWN_owner_4,deal_application_thread_id,completion_status
0,900,4813,606.0,A,150000.0,100.0,NaN,NaN,NaN,NaN,...,0.00,NaN,preferred,NaN,606.0,A,150000.0,100.0,4642.0,Paid off with renewal
1,329,1406,703.0,A,150000.0,100.0,NaN,NaN,NaN,NaN,...,107027.25,NaN,preferred,NaN,703.0,A,150000.0,100.0,1362.0,Paid in full
2,1035,7415,692.0,A,150000.0,100.0,NaN,NaN,NaN,NaN,...,0.00,NaN,preferred,NaN,692.0,A,150000.0,100.0,6485.0,Default
3,1670,6759,684.0,A,150000.0,100.0,NaN,NaN,NaN,NaN,...,0.00,NaN,preferred,NaN,684.0,A,150000.0,100.0,6035.0,Paid via discounted payoff
4,1726,5867,625.0,A,150000.0,100.0,NaN,NaN,NaN,NaN,...,0.00,NaN,preferred,NaN,625.0,A,150000.0,100.0,5299.0,Paid in full


In [9]:
dataML.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1810 entries, 0 to 1809
Data columns (total 56 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Unnamed: 0                                1810 non-null   int64  
 1   id                                        1810 non-null   int64  
 2   owner_1_score                             1786 non-null   float64
 3   RATE_owner_1                              1780 non-null   object 
 4   CAP_AMOUNT_owner_1                        1775 non-null   float64
 5   PERCENT_OWN_owner_1                       1793 non-null   float64
 6   owner_2_score                             214 non-null    float64
 7   RATE_owner_2                              214 non-null    object 
 8   CAP_AMOUNT_owner_2                        202 non-null    float64
 9   PERCENT_OWN_owner_2                       259 non-null    float64
 10  owner_3_score                       

In [10]:
# Calculate the percentage of missing values in df_read
percentage_missing = dataML.isnull().sum() / len(dataML) * 100

# Create a new DataFrame with columns from df and index set to None
new_df = pd.DataFrame(columns=dataML.columns, index=None)
pd.set_option('display.max_columns', None)

# Creating new Percentage index
new_df.loc['Percentage'] = percentage_missing.values
new_df

,Unnamed: 0,id,owner_1_score,RATE_owner_1,CAP_AMOUNT_owner_1,PERCENT_OWN_owner_1,owner_2_score,RATE_owner_2,CAP_AMOUNT_owner_2,PERCENT_OWN_owner_2,owner_3_score,RATE_owner_3,CAP_AMOUNT_owner_3,PERCENT_OWN_owner_3,years_in_business,RATE_ID_FOR_years_in_business,fsr,RATE_ID_FOR_fsr,location,RATE_ID_FOR_location,funded_last_30,RATE_ID_FOR_funded_last_30,judgement_lien_percent,RATE_ID_FOR_judgement_lien_percent,INPUT_VALUE_ID_FOR_judgement_lien_amount,RATE_ID_FOR_judgement_lien_amount,INPUT_VALUE_ID_FOR_judgement_lien_time,RATE_ID_FOR_judgement_lien_time,INPUT_VALUE_ID_FOR_num_negative_days,RATE_ID_FOR_num_negative_days,INPUT_VALUE_ID_FOR_num_deposits,RATE_ID_FOR_num_deposits,INPUT_VALUE_ID_FOR_monthly_gross,RATE_ID_FOR_monthly_gross,INPUT_VALUE_ID_FOR_average_ledger,RATE_ID_FOR_average_ledger,INPUT_VALUE_ID_FOR_fc_margin,RATE_ID_FOR_fc_margin,INPUT_VALUE_ID_FOR_tax_lien_amount,RATE_ID_FOR_tax_lien_amount,INPUT_VALUE_ID_FOR_tax_lien_percent,RATE_ID_FOR_tax_lien_percent,INPUT_VALUE_ID_FOR_tax_lien_count,RATE_ID_FOR_tax_lien_count,INPUT_VALUE_ID_FOR_current_position,RATE_ID_FOR_current_position,INPUT_VALUE_ID_FOR_avg_net_deposits,RATE_ID_FOR_avg_net_deposits,INPUT_VALUE_ID_FOR_industry_type,RATE_ID_FOR_industry_type,INPUT_VALUE_owner_4,RATE_owner_4,CAP_AMOUNT_owner_4,PERCENT_OWN_owner_4,deal_application_thread_id,completion_status
Percentage,0.0,0.0,1.325967,1.657459,1.933702,0.939227,88.176796,88.176796,88.839779,85.690608,99.447514,99.447514,99.447514,97.679558,0.883978,0.883978,27.348066,98.950276,0.607735,4.088398,0.0,99.668508,0.0,0.0,0.0,0.0,100.0,100.0,0.0,0.110497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.607735,0.607735,0.773481,0.773481,0.38674,100.0,1.657459,100.0,1.823204,4.364641,5.303867,3.038674,0.0,0.0


In [11]:
features_to_keep = dataML.columns[((dataML.isnull().sum()/len(dataML))*100 > 20)].to_list()
print("Total features before:",len(dataML.columns))
print("Total features now:",len(features_to_keep))

Total features before: 56
Total features now: 15


In [12]:
df1=dataML[features_to_keep]
df1.shape

(1810, 15)

In [13]:
df1.head()

,owner_2_score,RATE_owner_2,CAP_AMOUNT_owner_2,PERCENT_OWN_owner_2,owner_3_score,RATE_owner_3,CAP_AMOUNT_owner_3,PERCENT_OWN_owner_3,fsr,RATE_ID_FOR_fsr,RATE_ID_FOR_funded_last_30,INPUT_VALUE_ID_FOR_judgement_lien_time,RATE_ID_FOR_judgement_lien_time,RATE_ID_FOR_avg_net_deposits,RATE_ID_FOR_industry_type
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.0,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df1.columns

Index(['owner_2_score', 'RATE_owner_2', 'CAP_AMOUNT_owner_2',
       'PERCENT_OWN_owner_2', 'owner_3_score', 'RATE_owner_3',
       'CAP_AMOUNT_owner_3', 'PERCENT_OWN_owner_3', 'fsr', 'RATE_ID_FOR_fsr',
       'RATE_ID_FOR_funded_last_30', 'INPUT_VALUE_ID_FOR_judgement_lien_time',
       'RATE_ID_FOR_judgement_lien_time', 'RATE_ID_FOR_avg_net_deposits',
       'RATE_ID_FOR_industry_type'],
      dtype='object')

In [ ]:
df1.drop('Unnamed: 0', inplace=True, axis=1)

In [17]:
df1.drop('id', inplace=True, axis=1)

KeyError: "['id'] not found in axis"

In [ ]:
df1

In [ ]:
df1['location'].unique()

In [ ]:
#df1.drop('RATE_owner_4', inplace=True, axis=1)
#df1.drop('CAP_AMOUNT_owner_4', inplace=True, axis=1)
#df1.drop('PERCENT_OWN_owner_4', inplace=True, axis=1)

In [ ]:
df1

In [ ]:
df1.columns

In [ ]:
df1.drop('years_in_business', inplace=True, axis=1)
df1.drop('RATE_ID_FOR_location', inplace=True, axis=1)
df1.drop('judgement_lien_percent', inplace=True, axis=1)
df1.drop('INPUT_VALUE_ID_FOR_judgement_lien_amount', inplace=True, axis=1)
df1.drop('INPUT_VALUE_ID_FOR_num_negative_days', inplace=True, axis=1)
df1.drop('INPUT_VALUE_ID_FOR_num_deposits', inplace=True, axis=1)
df1.drop('INPUT_VALUE_ID_FOR_monthly_gross', inplace=True, axis=1)
df1.drop('INPUT_VALUE_ID_FOR_average_ledger', inplace=True, axis=1)
df1.drop('INPUT_VALUE_ID_FOR_fc_margin', inplace=True, axis=1)
df1.drop('INPUT_VALUE_ID_FOR_tax_lien_amount', inplace=True, axis=1)
df1.drop('INPUT_VALUE_ID_FOR_tax_lien_percent', inplace=True, axis=1)
df1.drop('INPUT_VALUE_ID_FOR_tax_lien_count', inplace=True, axis=1)
df1.drop('RATE_ID_FOR_current_position', inplace=True, axis=1)
df1.drop('deal_application_thread_id', inplace=True, axis=1)
#df1.drop('PERCENT_OWN_owner_1', inplace=True, axis=1)


In [ ]:
df1


In [ ]:
count = df1['funded_last_30'].value_counts()

In [ ]:
count


In [ ]:
df1.drop('funded_last_30', inplace=True, axis=1)


In [ ]:
#df1['Difference'] = df1['owner_1_score'] - df1['INPUT_VALUE_owner_4']

In [ ]:
#df1
df1.drop('INPUT_VALUE_owner_4', inplace=True, axis=1)
df1.drop('RATE_owner_4', inplace=True, axis=1)
df1.drop('CAP_AMOUNT_owner_4', inplace=True, axis=1)
df1.drop('PERCENT_OWN_owner_4', inplace=True, axis=1)


In [ ]:
#df1['Difference'].unique()

In [ ]:
df1

In [ ]:
count1 = df1['RATE_ID_FOR_tax_lien_amount'].value_counts()
count2 = df1['RATE_ID_FOR_tax_lien_percent'].value_counts()
count3 = df1['RATE_ID_FOR_tax_lien_count'].value_counts()


In [ ]:
count1

In [ ]:
count2

In [ ]:
count3

In [ ]:
df1.drop('RATE_ID_FOR_tax_lien_amount', inplace=True, axis=1)
df1.drop('RATE_ID_FOR_tax_lien_percent', inplace=True, axis=1)
df1.drop('RATE_ID_FOR_tax_lien_count', inplace=True, axis=1)

In [ ]:
df1


In [ ]:
count4 = df1['RATE_ID_FOR_judgement_lien_percent'].value_counts()
count5 = df1['RATE_ID_FOR_judgement_lien_amount'].value_counts()
count6 = df1['RATE_ID_FOR_num_negative_days'].value_counts()

In [ ]:
count4

In [ ]:
count5


In [ ]:
count6

In [ ]:
df1.drop('RATE_ID_FOR_judgement_lien_percent', inplace=True, axis=1)
df1.drop('RATE_ID_FOR_judgement_lien_amount', inplace=True, axis=1)

In [ ]:
df1


In [ ]:
count7 = df1['PERCENT_OWN_owner_1'].value_counts()

In [ ]:
count7

In [ ]:
df1.isnull().sum()

In [ ]:
ata_type = df1['INPUT_VALUE_ID_FOR_industry_type'].dtype
ata_type

In [ ]:
df1['owner_1_score'].fillna(df1['owner_1_score'].mean(), inplace=True)
df1['CAP_AMOUNT_owner_1'].fillna(df1['CAP_AMOUNT_owner_1'].mean(), inplace=True)
df1['PERCENT_OWN_owner_1'].fillna(df1['PERCENT_OWN_owner_1'].mean(), inplace=True)
df1['INPUT_VALUE_ID_FOR_avg_net_deposits'].fillna(df1['INPUT_VALUE_ID_FOR_avg_net_deposits'].mean(), inplace=True)
#df1['owner_1_score'].fillna(df1['owner_1_score'].mean(), inplace=True)
#df1['owner_1_score'].fillna(df1['owner_1_score'].mean(), inplace=True)


In [ ]:
df1.isnull().sum()

In [ ]:
#df1.isnull().sum()
#df1['owner_1_score'] = df1['owner_1_score'].astype(int)

In [ ]:
# Replace 'numerical_column' and 'categorical_column' with your column names
#numerical_column = 'owner_1_score'
#categorical_column = 'RATE_owner_1'

# Fill the numerical column with mean
#mean_value = df1[numerical_column].mean()
#df1[numerical_column].fillna(mean_value, inplace=True)

# Fill the categorical column based on conditions
#conditions = [
   # (df1[numerical_column] > 650),
    #(df1[numerical_column] > 600) & (df1[numerical_column] <= 650),
    #(df1[numerical_column] > 500) & (df1[numerical_column] <= 600),
   # (df1[numerical_column] <= 500)
#]

#choices = ['A', 'B', 'C', 'D']

#df1[categorical_column] = df1[categorical_column].fillna(pd.Series(pd.Categorical.from_codes(
#    pd.np.select(conditions, choices),
  #  categories=choices
#)))

# Save the modified dataset
#dataset.to_csv('filled_dataset.csv', index=False)

In [ ]:
df1['completion_status'] = df1['completion_status'].replace(['Paid off with renewal', 'Paid in full'], 'Paid in full')
df1['completion_status'] = df1['completion_status'].replace(['Paid via discounted payoff', 'Default'], 'Default')

In [ ]:
df1


In [ ]:
df1['RATE_owner_1'].fillna(method='ffill', inplace=True)
df1['RATE_ID_FOR_years_in_business'].fillna(method='ffill', inplace=True)
df1['location'].fillna(method='ffill', inplace=True)
df1['RATE_ID_FOR_num_negative_days'].fillna(method='ffill', inplace=True)
df1['INPUT_VALUE_ID_FOR_current_position'].fillna(method='ffill', inplace=True)
df1['INPUT_VALUE_ID_FOR_industry_type'].fillna(method='ffill', inplace=True)


In [ ]:
df1

In [ ]:
df1.isnull().sum()

In [ ]:
label_encoder=LabelEncoder()
df1['RATE_owner_1']=label_encoder.fit_transform(df1['RATE_owner_1'])
df1['RATE_ID_FOR_years_in_business']=label_encoder.fit_transform(df1['RATE_ID_FOR_years_in_business'])
df1['location']=label_encoder.fit_transform(df1['location'])
df1['RATE_ID_FOR_num_negative_days']=label_encoder.fit_transform(df1['RATE_ID_FOR_num_negative_days'])
df1['RATE_ID_FOR_num_deposits']=label_encoder.fit_transform(df1['RATE_ID_FOR_num_deposits'])
df1['RATE_ID_FOR_monthly_gross']=label_encoder.fit_transform(df1['RATE_ID_FOR_monthly_gross'])
df1['RATE_ID_FOR_average_ledger']=label_encoder.fit_transform(df1['RATE_ID_FOR_average_ledger'])
df1['RATE_ID_FOR_fc_margin']=label_encoder.fit_transform(df1['RATE_ID_FOR_fc_margin'])
df1['INPUT_VALUE_ID_FOR_industry_type']=label_encoder.fit_transform(df1['INPUT_VALUE_ID_FOR_industry_type'])
df1['completion_status']=label_encoder.fit_transform(df1['completion_status'])



In [ ]:
df1


In [ ]:
column_name = 'CAP_AMOUNT_owner_1'  # Replace with the actual column name
min_value = df1[column_name].min()
max_value = df1[column_name].max()
df1[column_name] = (df1[column_name] - min_value) / (max_value - min_value)

column_name = 'INPUT_VALUE_ID_FOR_avg_net_deposits'  # Replace with the actual column name
min_value = df1[column_name].min()
max_value = df1[column_name].max()
df1[column_name] = (df1[column_name] - min_value) / (max_value - min_value)


column_name = 'owner_1_score'  # Replace with the actual column name
min_value = df1[column_name].min()
max_value = df1[column_name].max()
df1[column_name] = (df1[column_name] - min_value) / (max_value - min_value)

In [ ]:
df1


In [ ]:

np.random.seed(2)
n=len(df1)
indx=np.arange(n)
print(indx)
np.random.shuffle(indx)
print(indx)
df1=df1.iloc[indx]

In [ ]:
df1

In [ ]:
#from sklearn.model_selection import GridSearchCV
#from sklearn.linear_model import Ridge

# Define the Ridge model
#model = Ridge()

# Define the grid of hyperparameters
#grid = {'alpha': np.arange(0, 1, 0.01)}

# Create the GridSearchCV object
#search = GridSearchCV(model, grid, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)

# Fit the feature extraction model to the data
#best_model = search.fit(df1, 'completion_status')

# Print the best mean squared error and best hyperparameters
#print('MSE: %.3f' % best_model.best_score_)
#print('Config: %s' % best_model.best_params_)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification



# Split into features (X) and labels (y)
X = df1.iloc[:,: -1]
Y = df1['completion_status']

# Create a Random Forest classifier
clf = RandomForestClassifier()

# Train the classifier
clf.fit(X, Y)

# Get the feature importances
importances = clf.feature_importances_

# Sort the feature importances in descending order
indices = importances.argsort()[::-1]

# Print the feature ranking
print("Feature ranking:")
for i, idx in enumerate(indices):
    print(f"Feature {idx}: importance score = {importances[idx]}")


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification

# Split into features (X) and labels (y)
X = df1.iloc[:, :-1]
y = df1['completion_status']

# Create a Random Forest classifier
clf = RandomForestClassifier()

# Define the parameter grid for grid search
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')

# Fit the grid search to the data
grid_search.fit(X, y)

# Get the best estimator from the grid search
best_estimator = grid_search.best_estimator_

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

# Get the feature importances from the best estimator
importances = best_estimator.feature_importances_

# Sort the feature importances in descending order
indices = importances.argsort()[::-1]

# Print the feature ranking
print("Feature ranking:")
for i, idx in enumerate(indices):
    print(f"Feature {idx}: importance score = {importances[idx]}")

In [ ]:



# Split into features (X) and labels (y)
X1 = df1.iloc[:,: -1]
Y1 = df1['completion_status']


# Create a logistic regression classifier
clf = LogisticRegression()

# Perform Recursive Feature Elimination with cross-validation
rfecv = RFECV(estimator=clf, cv=5)
X_selected = rfecv.fit_transform(X1, Y1)

# Get the selected feature indices
selected_indices = rfecv.get_support(indices=True)

# Print the selected feature indices
print("Selected feature indices:", selected_indices)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25,random_state=100)


In [ ]:
# MinMax Scaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
label='completion_status'

g = df1.groupby(label, group_keys=False)
balanced_data = pd.DataFrame(g.apply(lambda x: x.sample(g.size().min()))).reset_index(drop=True)
balanced_data.head()

In [ ]:
balanced_data.groupby('completion_status')['completion_status'].count()

In [ ]:
balanced_data

In [ ]:
#X=df1.drop('completion_status',axis=1)
#y=df1['completion_status']

Y = df1['completion_status']
X = df1.iloc[:,: -1]
X_train, X_test, y_train, y_test = train_test_split (X,Y, test_size=0.3)

# RandomForest

In [ ]:
#model1=RandomForestClassifier()
#model1.fit(X_train_ns,y_train_ns)

In [ ]:
#y_pred=model1.predict(X_test)
#print("Confusion Matrix: \n",confusion_matrix(y_test,y_pred))
#print('\n')
#print("Accuracy Score: \n",accuracy_score(y_test,y_pred))
#print('\n')
#print("Classification Report: \n",classification_report(y_test,y_pred))

# over sampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler
os=RandomOverSampler()
X_train_os,y_train_os=os.fit_resample(X_train,y_train)
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes after fit {}".format(Counter(y_train_os)))

In [ ]:
model2=RandomForestClassifier()
model2.fit(X_train_os,y_train_os)

y_pred=model2.predict(X_test)
print("Confusion Matrix: \n",confusion_matrix(y_test,y_pred))
print('\n')
print("Accuracy Score: \n",accuracy_score(y_test,y_pred))
print('\n')
print("Classification Report: \n",classification_report(y_test,y_pred))

# smootk sampling

In [ ]:
smt=SMOTETomek()
X_train_smt,y_train_smt=smt.fit_resample(X_train,y_train)
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes after fit {}".format(Counter(y_train_smt)))

# RandomForest after smootk sampling

In [ ]:
model3=RandomForestClassifier()
model3.fit(X_train_smt,y_train_smt)
y_pred=model3.predict(X_test)
print("Confusion Matrix: \n",confusion_matrix(y_test,y_pred))
print('\n')
print("Accuracy Score: \n",accuracy_score(y_test,y_pred))
print('\n')
print("Classification Report: \n",classification_report(y_test,y_pred))

# DecisionTreeClassifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier


# Create a Decision Tree classifier
classifier = DecisionTreeClassifier()

# Train the classifier
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Print the confusion matrix
print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))
print('\n')

# Print the accuracy score
print("Accuracy Score: \n", accuracy_score(y_test, y_pred))
print('\n')

# Print the classification report
print("Classification Report: \n", classification_report(y_test, y_pred))


# logistic

In [ ]:



# Create a Logistic Regression classifier
classifier = LogisticRegression()

# Train the classifier
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
# Print the confusion matrix
print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))
print('\n')

# Print the accuracy score
print("Accuracy Score: \n", accuracy_score(y_test, y_pred))
print('\n')

# Print the classification report
print("Classification Report: \n", classification_report(y_test, y_pred))


# svm

In [ ]:

# Create an SVM classifier
classifier = SVC()

# Train the classifier
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
# Print the confusion matrix
print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))
print('\n')

# Print the accuracy score
print("Accuracy Score: \n", accuracy_score(y_test, y_pred))
print('\n')

# Print the classification report
print("Classification Report: \n", classification_report(y_test, y_pred))



# GradientBoostingClassifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Create a Gradient Boosting Classifier
classifier = GradientBoostingClassifier()

# Train the classifier
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)

# Print the confusion matrix
print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))
print('\n')

# Print the accuracy score
print("Accuracy Score: \n", accuracy_score(y_test, y_pred))

prc2 = precision_score(y_test, y_pred)
rcl2 = recall_score(y_test, y_pred)
f1_2 = f1_score(y_test, y_pred)


print(prc2)
print(rcl2)
print(f1_2)
print('\n')

# Print the classification report
print("Classification Report: \n", classification_report(y_test, y_pred))


# lightgbm

In [ ]:
!pip install lightgbm

In [ ]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Create a LightGBM classifier
classifier = lgb.LGBMClassifier()

# Train the classifier
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)

# Print the confusion matrix
print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))
print('\n')

# Print the accuracy score
print("Accuracy Score: \n", accuracy_score(y_test, y_pred))
print('\n')

# Print the classification report
print("Classification Report: \n", classification_report(y_test, y_pred))


# AdaBoostClassifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Create an AdaBoost classifier
classifier = AdaBoostClassifier()

# Train the classifier
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)

# Print the confusion matrix
print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))
print('\n')

# Print the accuracy score
print("Accuracy Score: \n", accuracy_score(y_test, y_pred))
print('\n')

# Print the classification report
print("Classification Report: \n", classification_report(y_test, y_pred))

# KNeighborsClassifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Create a KNN classifier
classifier = KNeighborsClassifier()

# Train the classifier
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)

# Print the confusion matrix
print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))
print('\n')

# Print the accuracy score
print("Accuracy Score: \n", accuracy_score(y_test, y_pred))
print('\n')

# Print the classification report
print("Classification Report: \n", classification_report(y_test, y_pred))


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

# Define the build_model function for Keras Tuner
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 25)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

# Define the tuner and perform hyperparameter tuning
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=3,
    executions_per_trial=3,
    directory='my_tuning_dir',
    project_name='my_tuning_project'
)
Y = df1['completion_status']
X = df1.iloc[:,: -1]
X_train, X_test, y_train, y_test = train_test_split (X,Y, test_size=0.3)


tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Get the best hyperparameter configuration
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hyperparameters)

# Train the best model with the optimal hyperparameters
best_model.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test))

# Evaluate the best model
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Load the test data
test_data = pd.read_csv('test_data.csv')  # Replace 'test_data.csv' with your actual test data file

# Split the data into features and labels
X_test = test_data['text_column']  # Replace 'text_column' with the actual column name for your text data
y_test = test_data['label_column']  # Replace 'label_column' with the actual column name for your labels

# Preprocessing Techniques

# Technique 1: Tokenization and Vectorization using TF-IDF
vectorizer = TfidfVectorizer()
X_test_tfidf = vectorizer.fit_transform(X_test)

# Technique 2: Standardization of Numeric Features
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test)

# Technique 3: Other Preprocessing Techniques (Add your own preprocessing steps here)

# Evaluation Metrics

# Define a function to evaluate the model and print the metrics
def evaluate_model(y_true, y_pred):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("Accuracy:", accuracy)

# Evaluate Technique 1: Tokenization and Vectorization using TF-IDF
# Replace 'model' with your actual trained model
y_pred_tfidf = model.predict(X_test_tfidf)
print("Evaluation for Technique 1: Tokenization and Vectorization using TF-IDF")
evaluate_model(y_test, y_pred_tfidf)

# Evaluate Technique 2: Standardization of Numeric Features
# Replace 'model' with your actual trained model
y_pred_scaled = model.predict(X_test_scaled)
print("Evaluation for Technique 2: Standardization of Numeric Features")
evaluate_model(y_test, y_pred_scaled)

# Evaluate Technique 3: Other Preprocessing Techniques
# Add your own evaluation code here


In [ ]:
      # Storing my best hyperparameters in the best_h